#  RAG 성능 평가 📊

---


## 1. 개요 및 환경 설정

### 1.1 RAG 평가가 중요한 이유

**RAG(Retrieval-Augmented Generation)** 시스템은 외부 지식을 검색하여 LLM의 응답 품질을 향상시키는 기술입니다. 하지만 RAG 시스템의 성능을 객관적으로 측정하고 개선하기 위해서는 체계적인 평가가 필요합니다.

#### 주요 평가 목표:
- **검색 품질**: 관련성 높은 문서를 얼마나 잘 찾는가?
- **생성 품질**: 검색된 정보를 얼마나 잘 활용하는가?
- **종합 성능**: 사용자에게 얼마나 유용한 답변을 제공하는가?

<center>
<img src="https://raw.githubusercontent.com/tsdata/image_files/main/202505/rag_evaluation.png" alt="rag" align="center" border="0"  width="1000" height=auto>
</center>



### 1.2 환경 설정

In [1]:
# 필요한 패키지 설치
# pip install ragas langchain langchain-openai langchain-chroma python-dotenv

# 환경변수 설정
from dotenv import load_dotenv
import os
load_dotenv()

# 기본 라이브러리
import json
import pandas as pd
import numpy as np
from pprint import pprint
from glob import glob

# LangSmith 추적 설정 (선택사항)
print(f"LangSmith 추적: {os.getenv('LANGSMITH_TRACING')}")

LangSmith 추적: true


---

## 2. RAG 평가의 핵심 개념

### 2.1 평가 차원 (Evaluation Dimensions)

<div align="center">

| 평가 영역 | 세부 지표 | 설명 |
|-----------|-----------|------|
| **검색 (Retrieval)** | Context Relevancy | 검색된 문서가 질문과 얼마나 관련있는가? |
|  | Context Recall | 정답에 필요한 모든 정보가 검색되었는가? |
| **생성 (Generation)** | Faithfulness | 생성된 답변이 검색된 문서에 충실한가? |
|  | Answer Relevancy | 생성된 답변이 질문과 관련있는가? |
| **종합** | Answer Correctness | 생성된 답변이 정답과 일치하는가? |

</div>

### 2.2 평가 방법론

#### A. Reference-Free 평가
- **장점**: 정답 데이터 없이도 평가 가능
- **방법**: LLM-as-Judge 방식 활용
- **도구**: RAGAS, LangSmith 등

#### B. Reference-Based 평가
- **장점**: 객관적이고 일관된 평가
- **방법**: 정답과 비교하여 평가
- **지표**: BLEU, ROUGE, Semantic Similarity 등

---

## 3. RAGAS 프레임워크 소개

### 3.1 RAGAS란?

**RAGAS (Retrieval-Augmented Generation Assessment)** 는 RAG 시스템을 위한 오픈소스 평가 프레임워크입니다.

#### 주요 특징:
- ✅ **Reference-Free**: 정답 없이도 평가 가능
- ✅ **LLM-as-Judge**: GPT-4 등을 활용한 자동 평가
- ✅ **구성요소별 평가**: 검색과 생성을 개별적으로 평가
- ✅ **LangChain 통합**: 기존 RAG 파이프라인과 쉽게 연동

### 3.2 RAGAS 핵심 지표

```python
from ragas.metrics import (
    context_relevancy,      # 컨텍스트 관련성
    context_recall,         # 컨텍스트 회상률
    faithfulness,          # 충실도
    answer_relevancy,      # 답변 관련성
    answer_correctness     # 답변 정확성
)
```

#### 지표별 상세 설명:

1. **Context Relevancy (컨텍스트 관련성)**
   - 검색된 문서가 질문과 얼마나 관련있는지 측정
   - 계산식: `관련 문장 수 / 전체 문장 수`

2. **Context Recall (컨텍스트 검출률)**
   - 정답 생성에 필요한 정보가 얼마나 검색되었는지 측정
   - 정답(ground truth) 필요

3. **Faithfulness (충실도)**
   - 생성된 답변이 검색된 문서에 얼마나 충실한지 측정
   - 환각(hallucination) 검출에 중요

4. **Answer Relevancy (답변 관련성)**
   - 생성된 답변이 질문과 얼마나 관련있는지 측정

5. **Answer Correctness (답변 정확성)**
   - 생성된 답변이 정답과 얼마나 일치하는지 측정

---

## 4. 실습 1: 기본 RAG 시스템 구축

### 4.1 문서 준비 및 처리

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

def load_documents(file_paths):
    """텍스트 파일들을 로드하는 함수"""
    documents = []
    for path in file_paths:
        try:
            loader = TextLoader(path, encoding='utf-8')
            documents.extend(loader.load())
        except Exception as e:
            print(f"파일 로드 실패 {path}: {e}")
    return documents

# 예시: 한국어 문서 로드
korean_files = glob('./data/*_KR.md')
documents = load_documents(korean_files)

print(f"로드된 문서 수: {len(documents)}")
if documents:
    print(f"첫 번째 문서 미리보기:\n{documents[0].page_content[:200]}...")

로드된 문서 수: 2
첫 번째 문서 미리보기:
Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업체, 자동차 기술 및 야외 레크리에이션 회사입니다.

**주요 정보:**

- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국...


### 4.2 문서 분할 (Text Splitting)


In [3]:
# 한국어 텍스트에 최적화된 분할기 설정
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    separators=['\n\n', '\n', r'(?<=[.!?])\s+'],  # 문장 단위 분할
    chunk_size=300,           # 청크 크기
    chunk_overlap=30,         # 중복 영역
    is_separator_regex=True,  # 정규식 사용
    keep_separator=True       # 구분자 유지
)

# 문서 분할 실행
split_docs = text_splitter.split_documents(documents)

print(f"분할된 청크 수: {len(split_docs)}")
print(f"\n첫 번째 청크:")
print(f"메타데이터: {split_docs[0].metadata}")
print(f"내용: {split_docs[0].page_content}")

분할된 청크 수: 39

첫 번째 청크:
메타데이터: {'source': './data\\리비안_KR.md'}
내용: Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업체, 자동차 기술 및 야외 레크리에이션 회사입니다.

**주요 정보:**


### 4.3 벡터 스토어 생성

In [4]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# 임베딩 모델 초기화
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1536  # 선택사항: 차원 축소로 성능 향상
)

# Chroma 벡터 스토어 생성
vector_store = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding_model,
    collection_name="rag_evaluation_demo",
    persist_directory="./local_chroma_db",
    collection_metadata={'hnsw:space': 'cosine'}  # 코사인 유사도 사용
)

print(f"벡터 스토어에 저장된 문서 수: {len(vector_store.get()['ids'])}")

벡터 스토어에 저장된 문서 수: 39


### 4.4 RAG 체인 구성


In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# LLM 초기화
llm = ChatOpenAI(
    model="gpt-4.1-mini",  # 비용 효율적인 모델
    temperature=0,        # 일관된 결과를 위해 0으로 설정
)

# 검색기 설정
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # 상위 3개 문서 검색
)

# 프롬프트 템플릿
prompt_template = ChatPromptTemplate.from_template("""
다음 컨텍스트를 바탕으로 질문에 답하세요. 
컨텍스트에 없는 정보는 추측하지 마세요.

컨텍스트:
{context}

질문: {question}

답변:
""")

# RAG 체인 구성
def format_docs(docs):
    """문서 리스트를 문자열로 변환"""
    return "\n\n".join(doc.page_content for doc in docs)

def rag_chain(question: str) -> dict:
    """RAG 체인 실행 함수"""
    # 1. 문서 검색
    retrieved_docs = retriever.invoke(question)
    
    # 2. 컨텍스트 준비
    context = format_docs(retrieved_docs)
    
    # 3. LLM으로 답변 생성
    response = llm.invoke(
        prompt_template.format_prompt(
            context=context, 
            question=question
        )
    ).content
    
    return {
        "question": question,
        "context": context,
        "answer": response,
        "retrieved_docs": retrieved_docs
    }

# 테스트 실행
test_question = "테슬라가 설립된 연도는?"
result = rag_chain(test_question)

print(f"질문: {result['question']}")
print(f"답변: {result['answer']}")
print(f"검색된 문서 수: {len(result['retrieved_docs'])}")

질문: 테슬라가 설립된 연도는?
답변: 테슬라는 2003년에 설립되었습니다.
검색된 문서 수: 3


---

## 5. 평가 데이터셋 생성

### 5.1 수동 데이터셋 생성

In [7]:
# Rivian 평가 데이터셋 생성
rivian_evaluation_questions = [
    {
        "question": "Rivian이 설립된 연도와 설립자는 누구인가?",
        "ground_truth": "Rivian은 2009년 6월에 R. J. 스캐린지에 의해 설립되었다. 처음에는 Mainstream Motors로 설립되었으나 2011년에 Rivian Automotive로 사명을 변경했다."
    },
    {
        "question": "Rivian의 본사는 어디에 위치하고 있는가?",
        "ground_truth": "Rivian의 본사는 미국 캘리포니아 주 어바인(Irvine, California)에 위치하고 있다."
    },
    {
        "question": "Rivian의 2023년 생산량과 수익은 얼마인가?",
        "ground_truth": "2023년 Rivian의 생산량은 57,232대이고, 수익은 44억 3천만 미국 달러였다."
    },
    {
        "question": "Rivian의 주요 제품은 무엇인가?",
        "ground_truth": "Rivian의 주요 제품은 전기 자동차와 배터리이다. 주요 차량 모델로는 R1T(픽업 트럭), R1S(SUV), 전기 배달 밴(EDV), R2(소형 SUV), R3(소형 SUV) 등이 있다."
    },
    {
        "question": "R1T 모델의 특징은 무엇인가?",
        "ground_truth": "R1T는 4개의 전기 모터가 장착된 픽업 트럭이다. 배터리 크기는 105 kWh에서 180 kWh까지 다양하며, 2021년 말에 배송이 시작되어 Rivian을 완전 전기 픽업을 소비자 시장에 출시한 최초의 자동차 제조업체로 만들었다."
    },
    {
        "question": "Rivian의 주요 제조 공장은 어디에 있는가?",
        "ground_truth": "Rivian의 주요 제조 공장은 일리노이 주 노멀(Normal, Illinois)에 있다. 이 공장은 2017년에 이전 Mitsubishi Motors 제조 공장을 1,600만 달러에 인수한 것으로, 차량 부품 생산과 조립을 수행한다."
    },
    {
        "question": "Rivian의 IPO는 언제 이루어졌고 얼마를 조달했는가?",
        "ground_truth": "Rivian의 IPO는 2021년 11월에 이루어졌으며, 135억 달러를 조달하여 상장 회사가 되었다."
    },
    {
        "question": "Rivian의 2023년 순손실은 얼마인가?",
        "ground_truth": "Rivian의 2023년 순손실은 54억 미국 달러였다."
    },
    {
        "question": "R2 모델의 출시 예정일은 언제인가?",
        "ground_truth": "R2는 더 작고 저렴한 SUV로, 새로운 플랫폼에서 2026년 초에 출시될 예정이다."
    },
    {
        "question": "Rivian의 충전 네트워크 이름은 무엇이고 언제 시작되었는가?",
        "ground_truth": "Rivian의 충전 네트워크는 Rivian Adventure Network이다. 2022년에 미국에서 충전 네트워크를 시작했으며, 2024년에 다른 차량에도 개방했다."
    },
    {
        "question": "Volkswagen과 Rivian의 파트너십 내용은 무엇인가?",
        "ground_truth": "2024년 6월, Volkswagen Group은 전기 아키텍처 및 소프트웨어 기술 개발을 목표로 Rivian에 최대 50억 달러를 투자할 의향을 발표했다."
    },
    {
        "question": "Rivian의 2023년 직원 수는 몇 명인가?",
        "ground_truth": "2023년 12월 기준으로 Rivian의 직원 수는 16,790명이다."
    },
    {
        "question": "Rivian이 NACS(북미 충전 표준)를 채택한다고 발표한 시기는 언제인가?",
        "ground_truth": "Rivian은 2023년 6월에 Tesla의 북미 충전 시스템(NACS) 채택을 발표했으며, 2025년 모델 연도부터 북미에서 차량에 NACS를 채택할 예정이다."
    },
    {
        "question": "Electric Delivery Van (EDV)은 주로 어떤 용도로 설계되었는가?",
        "ground_truth": "Electric Delivery Van (EDV)은 상업용 전기 밴으로, 주로 Amazon용으로 설계되어 사용되고 있다."
    },
    {
        "question": "Rivian의 2020년부터 2023년까지 수익 성장을 설명하시오.",
        "ground_truth": "Rivian의 수익은 2020년 0달러에서 시작하여, 2021년 5,500만 달러, 2022년 16억 5,800만 달러, 2023년 44억 3,400만 달러로 급격한 성장을 보였다."
    },
    {
        "question": "Rivian의 최대 주주들은 누구인가?",
        "ground_truth": "2023년 12월 현재 Rivian의 최대 주주는 Amazon, T. Rowe Price International, The Vanguard Group, BlackRock 및 Fidelity Investments였다."
    },
    {
        "question": "Rivian이 직면한 주요 소송들은 무엇인가?",
        "ground_truth": "Rivian은 여러 소송에 직면했다. 2020년 7월 Tesla가 독점 정보 도용과 직원 영입을 주장하며 소송을 제기했고, 2021년 3월 Illinois Automobile Dealers Association이 직접 판매에 대해 소송했으며, 2021년 11월 전 VP Laura Schwab이 차별 혐의로 소송을 제기했다."
    },
    {
        "question": "Rivian이 2024년에 발표한 인력 감축 계획은 무엇인가?",
        "ground_truth": "2024년 2월, Rivian은 급여 직원 10% 감축을 발표했다. 이전에도 2022년 7월에 6%의 인력 감축을 발표한 바 있다."
    },
    {
        "question": "Rivian의 서비스 지역과 거래소 정보는 무엇인가?",
        "ground_truth": "Rivian의 서비스 지역은 북미이며, NASDAQ에서 RIVN 티커로 거래되는 상장회사이다."
    },
    {
        "question": "R3 모델에 대해 알려진 정보는 무엇인가?",
        "ground_truth": "R3는 2024년 3월에 공개된 출시 예정인 전기 소형 SUV이다. 더 작은 가격대의 R2 SUV와 함께 발표되었지만, 구체적인 출시 일정은 명시되지 않았다."
    }
]

print(f"생성된 Rivian 평가 질문 수: {len(rivian_evaluation_questions)}")

생성된 Rivian 평가 질문 수: 20


In [8]:
# Tesla 평가 데이터셋 생성
tesla_evaluation_questions = [
    {
        "question": "Tesla는 언제 누구에 의해 설립되었는가?",
        "ground_truth": "Tesla는 2003년 7월 1일에 Martin Eberhard와 Marc Tarpenning에 의해 Tesla Motors로 설립되었다. Nikola Tesla를 기리기 위해 명명되었다."
    },
    {
        "question": "Elon Musk는 언제 Tesla의 CEO가 되었는가?",
        "ground_truth": "Elon Musk는 2004년 Tesla의 초기 자금 조달을 주도하여 2008년 10월에 CEO가 되었다."
    },
    {
        "question": "Tesla의 첫 번째 양산 차량은 무엇인가?",
        "ground_truth": "Tesla의 첫 번째 양산 차량은 Roadster로, 2008년에 생산이 시작되었다."
    },
    {
        "question": "Model S는 언제 출시되었고 어떤 특징이 있는가?",
        "ground_truth": "Model S는 2012년 6월에 출시된 고급 세단이다. 리프트백 차체 스타일과 듀얼 모터, 전륜 구동 레이아웃을 갖추고 있으며, 여러 자동차 상을 받았고 노르웨이와 전 세계에서 가장 많이 팔린 전기 자동차가 되었다."
    },
    {
        "question": "Tesla는 언제 IPO를 진행했고 얼마를 조달했는가?",
        "ground_truth": "Tesla는 2010년 6월 NASDAQ에 상장하여 2억 2,600만 달러를 조달했다."
    },
    {
        "question": "Tesla의 주요 제조 공장들은 어디에 위치하고 있는가?",
        "ground_truth": "Tesla의 주요 제조 공장들은 다음과 같다: Tesla 프리몬트 공장(캘리포니아, 2010), Gigafactory 네바다(2016), Gigafactory 뉴욕(2017), Gigafactory 상하이(2019), Gigafactory 베를린(2022), Gigafactory 텍사스(2022)이다."
    },
    {
        "question": "Model 3의 개발과 특징은 무엇인가?",
        "ground_truth": "Model 3는 2016년 4월에 공개된 중형차로, 일주일 만에 325,000건 이상의 예약이 접수되었다. '생산 지옥'으로 묘사된 생산 문제로 지연이 발생했지만, 2018년 말까지 세계에서 가장 많이 팔린 전기 자동차(2018-2021)가 되었다."
    },
    {
        "question": "Tesla의 현재 차량 라인업은 무엇인가?",
        "ground_truth": "2024년 11월 현재 Tesla는 Model S, Model X, Model 3, Model Y, Semi 및 Cybertruck의 6가지 차량 모델을 제공한다."
    },
    {
        "question": "Tesla Autopilot은 언제 발표되었고 무엇인가?",
        "ground_truth": "Tesla Autopilot은 2014년에 발표된 운전자 지원 시스템이다. Tesla에서 개발한 고급 운전자 지원 시스템(ADAS)으로, 부분적인 차량 자동화를 의미한다."
    },
    {
        "question": "Tesla Energy는 어떻게 설립되었는가?",
        "ground_truth": "Tesla Energy는 2016년 11월 Tesla가 SolarCity를 26억 달러에 인수하여 설립되었다. 태양 에너지 생성 시스템과 배터리 에너지 저장 제품을 개발, 구축, 판매 및 설치한다."
    },
    {
        "question": "Tesla의 충전 네트워크는 무엇인가?",
        "ground_truth": "Tesla의 충전 네트워크는 Supercharger 네트워크와 Destination 충전 위치 네트워크가 있다. Supercharger는 2012년에 도입된 고전압 DC 급속 충전 네트워크이고, Destination 충전은 호텔, 레스토랑 및 쇼핑 센터에 있는 더 느린 충전기이다."
    },
    {
        "question": "Model Y는 언제 출시되었고 어떤 특징이 있는가?",
        "ground_truth": "Model Y는 2019년 3월에 소개된 중형 크로스오버 SUV로, 배송은 2020년 3월에 시작되었다. 싱글 모터, 후륜 구동 또는 듀얼 모터, 전륜 구동 레이아웃을 갖추고 5인승 및 7인승 구성으로 제공되며, 고급 Model X SUV보다 저렴하도록 설계되었다."
    },
    {
        "question": "Tesla는 2023년에 배터리 전기 자동차 시장에서 몇 퍼센트의 점유율을 차지했는가?",
        "ground_truth": "2023년에 Tesla는 배터리 전기 자동차 시장에서 가장 큰 점유율인 19.9%를 차지했다."
    },
    {
        "question": "Semi 트럭의 특징은 무엇인가?",
        "ground_truth": "Tesla Semi는 클래스 8 세미 트럭으로, 트리 모터, 후륜 구동 레이아웃을 갖추고 있다. Tesla는 Semi가 일반적인 디젤 세미 트럭보다 약 3배 더 강력하고 주행 거리가 500마일(800km)이라고 주장한다. 초기 배송은 2022년 12월 1일에 PepsiCo에 이루어졌다."
    },
    {
        "question": "Cybertruck은 언제 발표되고 배송이 시작되었는가?",
        "ground_truth": "Cybertruck은 2019년 11월에 처음 발표된 풀사이즈 픽업 트럭으로, 2023년 11월에 배송이 시작되었다. 후륜 구동, 듀얼 모터 전륜 구동, 트리 모터 전륜 구동의 세 가지 모델이 제공된다."
    },
    {
        "question": "Tesla가 Bitcoin에 투자한 시기와 규모는 어떻게 되는가?",
        "ground_truth": "2021년 초, Tesla는 Bitcoin에 15억 달러를 투자했다. 환경 문제로 인해 잠시 결제 수단으로 허용하다가 중단했으며, 2022년 7월까지 Bitcoin 보유량의 약 75%를 매각했다."
    },
    {
        "question": "북미 충전 표준(NACS)이란 무엇인가?",
        "ground_truth": "북미 충전 표준(NACS)은 Tesla에서 개발한 전기 자동차 충전 커넥터 시스템이다. 2023년 5월과 2024년 2월 사이에 북미 EV 제조업체들이 Tesla의 북미 충전 표준으로 전환할 계획을 발표했다."
    },
    {
        "question": "Tesla의 로봇 공학 기술은 무엇인가?",
        "ground_truth": "Tesla는 대형 주조 기계(Giga Press)를 사용하여 대형 단일 피스 하체를 만든다. 또한 2022년부터 Optimus라는 휴머노이드 로봇을 개발해 왔다."
    },
    {
        "question": "Tesla의 비즈니스 전략은 무엇인가?",
        "ground_truth": "Tesla의 전략은 배터리 비용을 줄이기 위해 고가, 소량 차량으로 시작한 다음 더 저렴하고 대량 차량을 제공하는 것이다. Tesla는 자동차의 하드웨어를 지속적으로 업데이트하고 웹사이트와 회사 소유 매장을 통해 직접 차량을 판매하며, 수직적으로 통합되어 많은 구성 요소를 자체 개발한다."
    },
    {
        "question": "Tesla가 최근 발표한 미래 차량들은 무엇인가?",
        "ground_truth": "Tesla가 최근 발표한 미래 차량들은 다음과 같다: Roadster 2세대(2025년 출시 예정), Tesla 차세대 차량(2025년 상반기 배송 예정), Cybercab(2026년 출시 예정인 2인승 자율 주행차), Robovan(미래 개발을 위해 계획된 전기 자율 밴)이다."
    }
]

print(f"생성된 Tesla 평가 질문 수: {len(tesla_evaluation_questions)}")

생성된 Tesla 평가 질문 수: 20


### 5.2 RAGAS 자동 데이터셋 생성

In [10]:
from ragas.testset import TestsetGenerator
from ragas.testset.persona import Persona
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

# LLM과 임베딩 래퍼 설정
generator_llm = LangchainLLMWrapper(
    ChatOpenAI(model="gpt-4.1-mini", temperature=0.2)
)
generator_embeddings = LangchainEmbeddingsWrapper(
    OpenAIEmbeddings(model="text-embedding-3-small")
)

# 페르소나 정의 (다양한 관점에서 질문 생성)
personas = [
    Persona(
        name="graduate_researcher",  # 박사과정 연구원: 심도 있는 분석적 질문
        role_description="미국 전기차 시장을 연구하는 한국인 박사과정 연구원으로, 전기차 정책과 시장 동향에 대해 깊이 있는 분석을 하고 있습니다. 반드시, **한국어**를 사용합니다.",
    ),
    Persona(
        name="masters_student",    # 석사과정 학생: 개념 이해를 위한 질문
        role_description="전기차 산업을 공부하는 한국인 석사과정 학생으로, 미국 전기차 시장의 기초적인 개념과 트렌드를 이해하려 노력하고 있습니다. 반드시, **한국어**를 사용합니다.",
    ),
    Persona(
        name="industry_analyst",   # 산업 분석가: 실무 중심적 질문
        role_description="한국 자동차 회사에서 미국 전기차 시장을 분석하는 주니어 연구원으로, 실무적인 시장 데이터와 경쟁사 동향에 관심이 많습니다. 반드시, **한국어**를 사용합니다.",
    )
]

# 테스트셋 생성기 초기화
testset_generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    persona_list=personas
)

# 합성 데이터셋 생성
synthetic_dataset = testset_generator.generate_with_langchain_docs(
    documents=split_docs,
    testset_size=20,
)

Applying SummaryExtractor:  91%|█████████▏| 32/35 [00:04<00:00,  8.21it/s]unable to apply transformation: Invalid json output: From 2020 to 2023, the company's revenue grew from 0 to 4,434 million USD, while net losses varied, peaking at -6,752 million USD in 2022 and improving to -5,432 million USD in 2023. Total assets reached their highest point in 2021 at 22,294 million USD before declining to 16,778 million USD in 2023. As of December 2023, major shareholders include Amazon, T. Rowe Price International, The Vanguard Group, BlackRock, and Fidelity Investments. The company has partnerships with Alex Honnold, the Honnold Foundation, Casa Pueblo, Ewan McGregor, Charley Boorman, Yakima, and MAXTRAX. Additionally, the company is involved in ongoing litigation.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Applying CustomNodeFilter:   0%|          | 0/39 [00:00<?, ?it/s]         Node 675fe0c4-ce7b-43f2-89cd-b6f2aa1f9e2e does 

In [11]:
# 결과 확인
df = synthetic_dataset.to_pandas()
print(f"생성된 합성 데이터셋 크기: {len(df)}")
print(f"\n컬럼: {list(df.columns)}")
print(f"\n첫 번째 샘플:")
print(f"질문: {df.iloc[0]['user_input']}")
print(f"정답: {df.iloc[0]['reference']}")

# CSV로 저장
df.to_csv('./data/synthetic_testset.csv', index=False, encoding='utf-8')


생성된 합성 데이터셋 크기: 20

컬럼: ['user_input', 'reference_contexts', 'reference', 'synthesizer_name']

첫 번째 샘플:
질문: Rivian Automotive는 미국에서 어떤 회사인가요?
정답: Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업체이자 자동차 기술 및 야외 레크리에이션 회사입니다.


In [12]:
df

,user_input,reference_contexts,reference,synthesizer_name
0,Rivian Automotive는 미국에서 어떤 회사인가요?,"[Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동...","Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차...",single_hop_specifc_query_synthesizer
1,미국 전기 자동차 시장에서 NASDAQ에 상장된 RIVN 회사는 어떤 제품과 서비스...,[- **회사 유형:** 상장\n- **거래소:** NASDAQ: RIVN\n- *...,"RIVN은 미국 캘리포니아 주 어바인에 본사를 둔 전기 자동차 회사로, 전기 자동차...",single_hop_specifc_query_synthesizer
2,2023년 12월 직원 수 몇 명이야?,[- **총 자산 (2023):** 168억 미국 달러\n- **총 자본 (2023...,"2023년 12월 기준으로 직원 수는 16,790명입니다.",single_hop_specifc_query_synthesizer
3,영국에 Rivian 회사 시설 있나요? 어디에 위치해요? 그리고 이게 미국 전기차 ...,"[**개요**\n\nRivian은 ""스케이트보드"" 플랫폼(R1T 및 R1S 모델)을...","Rivian은 미국 일리노이 주 노멀에 생산 공장을 두고 있으며, 다른 시설들은 미...",single_hop_specifc_query_synthesizer
4,Mitsubshi Motors는 언제 미국에서 공장을 인수했나요?,[**역사**\n\n**초창기 (2009–15):**\n\n- 2009년 R. J....,Mitsubishi Motors의 이전 제조 공장은 2017년에 일리노이 주 노멀에...,single_hop_specifc_query_synthesizer
5,R1S 가격 인상 언제였나요?,[- R1T 배송은 2021년 9월에 시작되어 Rivian은 완전 전기 픽업을 소비...,"2022년 3월, Rivian은 부품 비용으로 인해 R1T 및 R1S의 가격 인상을...",single_hop_specifc_query_synthesizer
6,2022년부터 2024년까지 Tesla가 미국 전기차 시장에서 진행한 주요 전략적 ...,"[- 2022년 10월, 느슨한 토크 볼트로 인해 13,000대의 차량을 자발적으로...","2022년 10월 Tesla는 느슨한 토크 볼트 문제로 인해 13,000대 차량을 ...",single_hop_specifc_query_synthesizer
7,Amazon 왜 Rivian 전기 Delivery Van 많이 쓰고 있나 그리고 이...,"[**Volkswagen과의 파트너십 (2024)**\n\n- 2024년 6월, V...",Amazon용으로 설계되어 사용되는 Electric Delivery Van(EDV)...,single_hop_specifc_query_synthesizer
8,Rivian은 미국 전기차 시장에서 어떤 EV 충전 네트워크와 시스템을 운영하고 있나요?,[**EV 충전**\n\nRivian은 미국과 캐나다 전역에 공공 충전소 네트워크를...,"Rivian은 미국과 캐나다 전역에 공공 충전소 네트워크를 개발하고 있으며, 202...",single_hop_specifc_query_synthesizer
9,"미국 전기차 회사 Rivian의 Palo Alto, California 시설은 어떤...","[**시설**\n\n- **Irvine, California:** 차량 엔지니어링 ...","Rivian의 Palo Alto, California 시설은 소프트웨어 개발 및 엔...",single_hop_specifc_query_synthesizer


### 5.3 평가용 데이터셋 준비

In [13]:
from ragas import EvaluationDataset

def create_evaluation_dataset(questions_data, rag_chain_func):
    """평가용 데이터셋을 생성하는 함수"""
    
    evaluation_data = []
    
    # 각 질문에 대해 RAG 체인을 실행하고 평가 데이터 구성
    for item in questions_data:
        if "question" in item: # 수동 데이터셋의 경우
            question = item["question"]
            ground_truth = item.get("ground_truth", "")
        else: # 합성 데이터셋의 경우
            question = item["user_input"]
            ground_truth = item["reference"]

        # RAG 체인 실행
        result = rag_chain_func(question)
        
        # 평가 데이터 구성
        eval_sample = {
            "user_input": question,
            "response": result["answer"],
            "retrieved_contexts": [doc.page_content for doc in result["retrieved_docs"]],
            "reference": ground_truth
        }
        
        evaluation_data.append(eval_sample)
        
        print(f"처리 완료: {question[:50]}...")
    
    return EvaluationDataset.from_list(evaluation_data)

# 평가 데이터셋 준비 : 옵션 1 - 수동 데이터셋 사용
# evaluation_questions = rivian_evaluation_questions + tesla_evaluation_questions

# 평가 데이터셋 준비 : 옵션 2 - 합성 데이터셋 사용
evaluation_questions = synthetic_dataset.to_list()  # 합성 데이터셋을 평가 질문으로 사용

print(f"총 {len(evaluation_questions)}개의 평가 질문이 있습니다.")

총 20개의 평가 질문이 있습니다.


In [14]:
# 평가 데이터셋 생성
eval_dataset = create_evaluation_dataset(evaluation_questions, rag_chain) 

# 데이터프레임으로 확인
eval_df = eval_dataset.to_pandas()
print(f"\n평가 데이터셋 생성 완료:")
print(f"- 샘플 수: {len(eval_df)}")
print(f"- 컬럼: {list(eval_df.columns)}")

# 저장
eval_df.to_csv('./data/evaluation_dataset.csv', index=False, encoding='utf-8')

처리 완료: Rivian Automotive는 미국에서 어떤 회사인가요?...
처리 완료: 미국 전기 자동차 시장에서 NASDAQ에 상장된 RIVN 회사는 어떤 제품과 서비스를 제공...
처리 완료: 2023년 12월 직원 수 몇 명이야?...
처리 완료: 영국에 Rivian 회사 시설 있나요? 어디에 위치해요? 그리고 이게 미국 전기차 시장에 ...
처리 완료: Mitsubshi Motors는 언제 미국에서 공장을 인수했나요?...
처리 완료: R1S 가격 인상 언제였나요?...
처리 완료: 2022년부터 2024년까지 Tesla가 미국 전기차 시장에서 진행한 주요 전략적 움직임과...
처리 완료: Amazon 왜 Rivian 전기 Delivery Van 많이 쓰고 있나 그리고 이거 어떤...
처리 완료: Rivian은 미국 전기차 시장에서 어떤 EV 충전 네트워크와 시스템을 운영하고 있나요?...
처리 완료: 미국 전기차 회사 Rivian의 Palo Alto, California 시설은 어떤 역할과...
처리 완료: Rivian이 2021년에 시작한 EV 충전기 설치와 2026년에 출시 예정인 R2 SUV...
처리 완료: Tesla Energy는 SolarCity 인수와 Model 3 생산 문제 어떻게 연결돼서...
처리 완료: Rivian이 2025년부터 북미 충전 시스템(NACS)을 채택하는 계획과 Tesla가 개...
처리 완료: Elon Musk 급여 패키지 거부 사건과 Tesla 초기 Roadster 개발 과정에서 ...
처리 완료: Elon Musk가 Tesla에서 어떤 역할을 했으며, 2024년 델라웨어 법원에서 그의 ...
처리 완료: Rivian 일리노이 주 노멀 공장 언제 인수했고, 그 공장에서 어떤 모델 생산 시작했나요...
처리 완료: Tesl가 2023년에 발표한 북미 충전 표준 NACS 채택과 관련된 주요 이벤트는 무엇이...
처리 완료: Rivian Automotive, Inc.는 언제 설립되었고, Rivian Automoti.

---

## 6. 실습 3: RAGAS 평가 수행

### 6.1 기본 평가 실행

In [15]:
from ragas import evaluate
from ragas.metrics import (
    LLMContextRecall,
    Faithfulness, 
    AnswerRelevancy,
    ContextPrecision,
    FactualCorrectness
)

# 평가용 LLM 설정
evaluator_llm = LangchainLLMWrapper(
    ChatOpenAI(model="gpt-4.1-mini", temperature=0)
)

# 평가 지표 선택
metrics = [
    LLMContextRecall(llm=evaluator_llm),           # 컨텍스트 검출율
    Faithfulness(llm=evaluator_llm),               # 충실도
    AnswerRelevancy(llm=evaluator_llm),            # 답변 관련성
    ContextPrecision(llm=evaluator_llm),           # 컨텍스트 정밀도
    FactualCorrectness(llm=evaluator_llm)          # 사실적 정확성
]

print("RAGAS 평가 시작...")

# 평가 실행
results = evaluate(
    dataset=eval_dataset,  # 처음 10개 샘플로 평가 (테스트 목적)
    metrics=metrics,
    llm=evaluator_llm,
    embeddings=generator_embeddings
)

print("평가 완료!")
print(f"\n전체 평가 결과:")
print(results)

RAGAS 평가 시작...


Evaluating: 100%|██████████| 100/100 [02:06<00:00,  1.26s/it]


평가 완료!

전체 평가 결과:
{'context_recall': 0.6333, 'faithfulness': 0.7164, 'answer_relevancy': 0.5596, 'context_precision': 0.6125, 'factual_correctness(mode=f1)': 0.6140}


### 6.2 상세 결과 분석


In [17]:
results_df

,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,answer_relevancy,context_precision,factual_correctness(mode=f1)
0,Rivian Automotive는 미국에서 어떤 회사인가요?,"[Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동...","Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차...","Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차...",1.000000,0.777778,0.797822,1.000000,0.67
1,미국 전기 자동차 시장에서 NASDAQ에 상장된 RIVN 회사는 어떤 제품과 서비스...,[- **회사 유형:** 상장\n- **거래소:** NASDAQ: RIVN\n- *...,RIVN으로 NASDAQ에 상장된 미국 전기 자동차 제조업체 Rivian Autom...,"RIVN은 미국 캘리포니아 주 어바인에 본사를 둔 전기 자동차 회사로, 전기 자동차...",1.000000,1.000000,0.696117,1.000000,0.89
2,2023년 12월 직원 수 몇 명이야?,[- **총 자산 (2023):** 168억 미국 달러\n- **총 자본 (2023...,"2023년 12월 직원 수는 16,790명입니다.","2023년 12월 기준으로 직원 수는 16,790명입니다.",1.000000,1.000000,0.691425,1.000000,1.00
3,영국에 Rivian 회사 시설 있나요? 어디에 위치해요? 그리고 이게 미국 전기차 ...,"[Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동...","네, Rivian은 영국에 Woking에 추가 사무실을 보유하고 있습니다. \n\n...","Rivian은 미국 일리노이 주 노멀에 생산 공장을 두고 있으며, 다른 시설들은 미...",0.000000,1.000000,0.000000,0.500000,0.40
4,Mitsubshi Motors는 언제 미국에서 공장을 인수했나요?,[Roadster 생산은 2008년에 시작되었습니다. 2009년 1월까지 Tesla...,Mitsubshi Motors에 대한 정보는 컨텍스트에 없습니다.,Mitsubishi Motors의 이전 제조 공장은 2017년에 일리노이 주 노멀에...,0.000000,1.000000,0.000000,0.000000,0.00
5,R1S 가격 인상 언제였나요?,[- R1T 배송은 2021년 9월에 시작되어 Rivian은 완전 전기 픽업을 소비...,R1S 가격 인상은 2022년 3월에 발표되었습니다.,"2022년 3월, Rivian은 부품 비용으로 인해 R1T 및 R1S의 가격 인상을...",1.000000,1.000000,0.691135,1.000000,0.50
6,2022년부터 2024년까지 Tesla가 미국 전기차 시장에서 진행한 주요 전략적 ...,"[## 비즈니스 전략\n\nTesla의 전략은 배터리 비용을 줄이기 위해 고가, 소...",2022년부터 2024년까지 Tesla가 미국 전기차(EV) 시장에서 진행한 주요 ...,"2022년 10월 Tesla는 느슨한 토크 볼트 문제로 인해 13,000대 차량을 ...",0.000000,0.516129,0.752363,0.000000,0.06
7,Amazon 왜 Rivian 전기 Delivery Van 많이 쓰고 있나 그리고 이...,"[**Volkswagen과의 파트너십 (2024)**\n\n- 2024년 6월, V...",Amazon이 Rivian의 전기 Delivery Van(EDV)을 많이 사용하는 ...,Amazon용으로 설계되어 사용되는 Electric Delivery Van(EDV)...,1.000000,0.625000,0.000000,0.833333,0.75
8,Rivian은 미국 전기차 시장에서 어떤 EV 충전 네트워크와 시스템을 운영하고 있나요?,[**EV 충전**\n\nRivian은 미국과 캐나다 전역에 공공 충전소 네트워크를...,Rivian은 미국과 캐나다 전역에 공공 충전소 네트워크인 Rivian Advent...,"Rivian은 미국과 캐나다 전역에 공공 충전소 네트워크를 개발하고 있으며, 202...",1.000000,0.750000,0.661702,1.000000,0.86
9,"미국 전기차 회사 Rivian의 Palo Alto, California 시설은 어떤...","[**시설**\n\n- **Irvine, California:** 차량 엔지니어링 ...","Rivian의 Palo Alto, California 시설은 소프트웨어 개발 및 엔...","Rivian의 Palo Alto, California 시설은 소프트웨어 개발 및 엔...",1.000000,1.000000,0.742735,1.000000,1.00


In [16]:
# 결과를 DataFrame으로 변환
results_df = results.to_pandas()

print(f"\n상세 평가 결과:")
print("="*80)

# 각 샘플별 상세 결과
for idx, row in results_df.iterrows():
    print(f"\n[샘플 {idx+1}]")
    print(f"질문: {row['user_input'][:100]}...")
    print(f"답변: {row['response'][:100]}...")
    
    # 지표별 점수 출력
    for col in results_df.columns:
        if col not in ['user_input', 'response', 'retrieved_contexts', 'reference']:
            if pd.notna(row[col]):
                print(f"  {col}: {row[col]:.3f}")
    print("-" * 50)


상세 평가 결과:

[샘플 1]
질문: Rivian Automotive는 미국에서 어떤 회사인가요?...
답변: Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업체이자 자동차 기술 및 야외 레크리에이션 회사입니다. 본사는 캘리포니아 주 어바인에 위치해...
  context_recall: 1.000
  faithfulness: 0.778
  answer_relevancy: 0.798
  context_precision: 1.000
  factual_correctness(mode=f1): 0.670
--------------------------------------------------

[샘플 2]
질문: 미국 전기 자동차 시장에서 NASDAQ에 상장된 RIVN 회사는 어떤 제품과 서비스를 제공하며, 2023년의 생산량과 재무 성과는 어떻게 되나요?...
답변: RIVN으로 NASDAQ에 상장된 미국 전기 자동차 제조업체 Rivian Automotive, Inc.는 전기 자동차와 배터리를 주요 제품으로 제공하며, 전기 자동차 충전과 자동차...
  context_recall: 1.000
  faithfulness: 1.000
  answer_relevancy: 0.696
  context_precision: 1.000
  factual_correctness(mode=f1): 0.890
--------------------------------------------------

[샘플 3]
질문: 2023년 12월 직원 수 몇 명이야?...
답변: 2023년 12월 직원 수는 16,790명입니다....
  context_recall: 1.000
  faithfulness: 1.000
  answer_relevancy: 0.691
  context_precision: 1.000
  factual_correctness(mode=f1): 1.000
------------------------------------------

In [18]:
# 통계 요약
print(f"\n📊 평가 지표 통계:")
print("="*50)

numeric_columns = results_df.select_dtypes(include=[np.number]).columns 
summary_stats = results_df[numeric_columns].describe()

for metric in numeric_columns:
    mean_score = summary_stats.loc['mean', metric]
    std_score = summary_stats.loc['std', metric]
    print(f"{metric}:")
    print(f"  평균: {mean_score:.3f} (±{std_score:.3f})")
    print(f"  범위: {summary_stats.loc['min', metric]:.3f} ~ {summary_stats.loc['max', metric]:.3f}")



📊 평가 지표 통계:
context_recall:
  평균: 0.633 (±0.400)
  범위: 0.000 ~ 1.000
faithfulness:
  평균: 0.716 (±0.256)
  범위: 0.267 ~ 1.000
answer_relevancy:
  평균: 0.560 (±0.299)
  범위: 0.000 ~ 0.838
context_precision:
  평균: 0.612 (±0.453)
  범위: 0.000 ~ 1.000
factual_correctness(mode=f1):
  평균: 0.614 (±0.274)
  범위: 0.000 ~ 1.000


In [19]:
# 결과 저장
results_df.to_csv('./data/ragas_evaluation_results.csv', index=False, encoding='utf-8')

### 6.3 성능 개선 분석

In [ ]:
def analyze_performance_issues(results_df):
    """성능 문제를 분석하고 개선 방안을 제시하는 함수"""
    
    print("🔍 성능 분석 및 개선 방안")
    print("="*60)
    
    # 1. 낮은 점수를 받은 샘플 식별
    low_performance_threshold = 0.6
    
    for metric in ['faithfulness', 'answer_relevancy', 'context_precision']:
        if metric in results_df.columns:
            low_scores = results_df[results_df[metric] < low_performance_threshold]
            
            if not low_scores.empty:
                print(f"\n⚠️  {metric}이 낮은 샘플들 ({len(low_scores)}개):")
                for idx, row in low_scores.iterrows():
                    print(f"  - 샘플 {idx}: {row[metric]:.3f}")
                    print(f"    질문: {row['user_input'][:80]}...")
                
                # 개선 방안 제시
                if metric == 'faithfulness':
                    print("\n  💡 개선 방안:")
                    print("     - 프롬프트에 '컨텍스트에만 기반하여 답변하세요' 강조")
                    print("     - Temperature를 더 낮게 설정")
                    print("     - 검색된 문서의 품질 확인")
                
                elif metric == 'answer_relevancy':
                    print("\n  💡 개선 방안:")
                    print("     - 검색 키워드 확장 전략 적용")
                    print("     - 쿼리 재작성(Query Rewriting) 기법 사용")
                    print("     - 더 정교한 프롬프트 엔지니어링")
                
                elif metric == 'context_precision':
                    print("\n  💡 개선 방안:")
                    print("     - 검색 파라미터 조정 (k값, similarity threshold)")
                    print("     - 리랭킹(Re-ranking) 기법 적용")
                    print("     - 문서 분할 전략 재검토")
    
    # 2. 전체적인 성능 요약
    print(f"\n📈 전체 성능 요약:")
    
    numeric_cols = results_df.select_dtypes(include=[np.number]).columns
    avg_scores = results_df[numeric_cols].mean()
    
    for metric, score in avg_scores.items():
        performance_level = "우수" if score >= 0.8 else "보통" if score >= 0.6 else "개선 필요"
        print(f"  {metric}: {score:.3f} ({performance_level})")

# 성능 분석 실행
analyze_performance_issues(results_df)

---

## 7. A/B 테스트를 통한 시스템 비교

In [ ]:
def compare_rag_systems(eval_questions, system_a, system_b, system_names=["System A", "System B"]):
    """두 RAG 시스템을 비교 평가하는 함수"""
    print(f"🔬 A/B 테스트: {system_names[0]} vs {system_names[1]}")
    print("="*60)
    
    # 각 시스템의 결과 생성
    results_a = create_evaluation_dataset(eval_questions, system_a)
    results_b = create_evaluation_dataset(eval_questions, system_b)
    
    # 평가 실행
    eval_a = evaluate(results_a, metrics=metrics, llm=evaluator_llm, embeddings=generator_embeddings)
    eval_b = evaluate(results_b, metrics=metrics, llm=evaluator_llm, embeddings=generator_embeddings)
    
    eval_a_dict = eval_a.to_pandas().mean(numeric_only=True).to_dict()
    eval_b_dict = eval_b.to_pandas().mean(numeric_only=True).to_dict()
    
    # 결과 비교
    comparison_results = {}
    for metric in eval_a_dict.keys():
        if isinstance(eval_a_dict[metric], (int, float)) and isinstance(eval_b_dict[metric], (int, float)):
            score_a = eval_a_dict[metric]
            score_b = eval_b_dict[metric]
            improvement = ((score_b - score_a) / score_a) * 100 if score_a != 0 else 0
            
            comparison_results[metric] = {
                system_names[0]: score_a,
                system_names[1]: score_b,
                'improvement_%': improvement
            }
            
            winner = system_names[1] if score_b > score_a else system_names[0]
            print(f"{metric}:")
            print(f"  {system_names[0]}: {score_a:.3f}")
            print(f"  {system_names[1]}: {score_b:.3f}")
            print(f"  승자: {winner} (개선: {improvement:+.1f}%)")
            print()
    
    return comparison_results

In [ ]:
# 다른 검색 파라미터를 가진 시스템 비교
def rag_chain_v2(question: str) -> dict:
    """개선된 RAG 체인 (더 많은 문서 검색)"""
    retriever_v2 = vector_store.as_retriever(search_kwargs={"k": 5})
    retrieved_docs = retriever_v2.invoke(question)
    context = format_docs(retrieved_docs)
    
    # 더 상세한 프롬프트
    detailed_prompt = """당신은 도움이 되는 AI 어시스턴트입니다. 
주어진 컨텍스트를 신중하게 분석하여 정확하고 유용한 답변을 제공하세요.

컨텍스트:
{context}

질문: {question}

지침:
1. 컨텍스트에 기반하여 답변하세요
2. 확실하지 않은 정보는 추측하지 마세요
3. 답변은 명확하고 구체적으로 작성하세요

답변:"""
    
    response = llm.invoke(
        detailed_prompt.format(context=context, question=question)
    ).content
    
    return {
        "question": question,
        "context": context,
        "answer": response,
        "retrieved_docs": retrieved_docs
    }

# 비교 실행
comparison = compare_rag_systems(
    evaluation_questions[:3],  # 샘플 3개만 사용
    rag_chain, 
    rag_chain_v2,
    ["기본 시스템", "개선된 시스템"]
)

---

## 8. 참고 자료

### 8.1 핵심 개념 요약

| 개념 | 설명 | 중요도 |
|------|------|--------|
| **RAG** | 검색 증강 생성 | ⭐⭐⭐⭐⭐ |
| **RAGAS** | RAG 자동 평가 프레임워크 | ⭐⭐⭐⭐⭐ |
| **LLM-as-Judge** | LLM을 평가자로 활용하는 기법 | ⭐⭐⭐⭐ |
| **Context Relevancy** | 검색된 문서의 관련성 | ⭐⭐⭐⭐ |
| **Faithfulness** | 답변의 충실도 (환각 방지) | ⭐⭐⭐⭐⭐ |

### 8.2 추천 학습 자료

1. **논문**
   - [RAGAS: Automated Evaluation of Retrieval Augmented Generation](https://arxiv.org/abs/2309.15217)
   - [Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks](https://arxiv.org/abs/2005.11401)

2. **공식 문서**
   - [RAGAS Documentation](https://docs.ragas.io/)
   - [LangChain RAG Tutorial](https://python.langchain.com/docs/tutorials/rag/)
   - [LangSmith Evaluation Guide](https://docs.smith.langchain.com/evaluation)

3. **실습 자료**
   - [RAGAS GitHub Repository](https://github.com/explodinggradients/ragas)
   - [LangChain RAG Examples](https://github.com/langchain-ai/langchain/tree/master/templates)

---
